In [1]:
from sqlalchemy import create_engine, MetaData

url = 'data/ecommerce.db'
engine = create_engine(f'sqlite:///{url}')

In [2]:
metadata_obj = MetaData()
metadata_obj.reflect(bind=engine)
metadata_obj.tables.keys()

dict_keys(['Clientes', 'Fornecedores', 'Funcionarios'])

In [3]:
import pandas as pd

df = pd.read_sql_table('Clientes', engine)
df

,ID_Cliente,Nome,Email,Rua_Numero,Estado,Valor_gasto
0,1,lucas moura,lucas.moura@email.com,"sitio de cardoso, 49",sao paulo,29017
1,2,julia andrade,julia.andrade@email.com,"trecho mariane teixeira, 90",tocantins,7834
2,3,rafael dias,rafael.dias@email.com,"setor de duarte, 114",paraiba,4071
3,4,carla souza,carla.souza@email.com,"recanto ana livia lopes, 53",para,17512
4,5,felipe neto,felipe.neto@email.com,"jardim de monteiro, 11",sao paulo,24307
...,...,...,...,...,...,...
95,96,ana luiza correia,heitor40@correia.net,"passarela pedro das neves, 806",para,9550
96,97,augusto nascimento,luiz-fernando91@da.br,"nucleo teixeira, 1",mato grosso do sul,22149
97,98,dr. caue pires,azevedomelissa@bol.com.br,ladeira peixoto,parana,33236
98,99,livia novaes,caldeiramelissa@costa.com,"travessa lima, 82",ceara,42653


In [4]:
df = pd.read_sql_table('Fornecedores', engine)
df

,ID_Fornecedores,Nome,Contato,Telefone,Email,Rua,Bairro,Cidade,Estado,cep
0,1,technex solutions,lucas martins,11912345678,info@technexsolutions.com,"rua das inovacoes, 123",centro,sao paulo,sp,1000000
1,2,quantum devices,isabela oliveira,11923456789,contact@quantumdevices.net,"avenida tecnologica, 456",jardim futuro,guarulhos,sp,20000000
2,3,innovatetech corporation,rafael pereira,11934567890,support@innovatetechcorp.com,"alameda do progresso, 789",centro das inovacoes,santo andre,sp,30000000
3,4,electrogadget innovations,ana silva,11945678901,sales@electrogadgetinnovations.com,"rua eletronica, 234",vila progresso,sao caetano,sp,40000000
4,5,nexustech systems,pedro almeida,11956789012,info@nexustechsystems.com,"travessa da conexao, 567",parque das maravilhas,sao paulo,sp,88000000
5,6,quantumware technologies,camila santos,11967890123,inquiries@quantumwaretech.com,"rua quantica, 890",bairro techville,guarulhos,sp,90000000


In [5]:
df = pd.read_sql_table('Funcionarios', engine)
df

,ID_Funcionarios,Nome,Cargo,DataContratacao,Telefone,Email,Rua,Bairro,Cidade,Estado,cep
0,1,carla souza,vendas,2022-03-15,1134567890,carla.souza@email.com,"rua das flores, 123",vila esperanca,sao paulo,sp,1000001
1,2,paulo silva,vendas,2022-05-10,1123456789,paulo.silva@email.com,"avenida do sol, 456",jardim da harmonia,sao paulo,sp,20000001
2,3,marta rocha,gerencia,2022-07-20,1145678901,marta.rochaa@email.com,"praca da liberdade, 789",parque dos sabias,sao paulo,sp,1000001
3,4,sofia ramos,tecnico,2022-01-12,1198765432,sofia.ramos@email.com,"alameda dos pinheiros, 234",floresta dos pinheiros,sao paulo,sp,40000001
4,5,joao pereira,estoque,2022-09-05,1123456789,joao.pereira@email.com,"travessa dos girassois, 567",centro,sao paulo,sp,20000001
5,6,talita borges,financeiro,2022-04-02,1156789012,talita.borges@email.com,"rodovia do mar, 890",nova aurora,sao paulo,sp,90000001
6,7,larissa freitas,vendas,2023-01-10,1187654321,larissa.freitas@email.com,"beco das estrelas, 1234",centro,sao paulo,sp,20000001


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [7]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

llama_model = 'llama-3.3-70b-versatile'
hf_embedings_model = 'BAAI/bge-m3'

Settings.llm = Groq(model=llama_model, api_key=GROQ_API_KEY)
Settings.embed_model = HuggingFaceEmbedding(model_name=hf_embedings_model)

/home/josiel/dev/work/python/llama-index-study/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping

sql_dtabase = SQLDatabase(
    engine=engine
)

table_node_map = SQLTableNodeMapping(
    sql_database=sql_dtabase
)

In [9]:
from llama_index.core.objects import SQLTableSchema

table_schema_obj = []

for table in metadata_obj.tables.keys():
    table_schema_obj.append(SQLTableSchema(table_name=table))


In [10]:
from llama_index.core.objects import ObjectIndex
from llama_index.core import VectorStoreIndex

obj_index = ObjectIndex.from_objects(table_schema_obj, table_node_map, VectorStoreIndex)

In [11]:
obj_retriever = obj_index.as_retriever(similarity_top_k=1)

In [12]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

query_engine = SQLTableRetrieverQueryEngine(sql_database=sql_dtabase, table_retriever=obj_retriever)

In [13]:
response = query_engine.query('Quais os estados mais frequentes na tabela de clientes?')
print(response)

Os estados mais frequentes na tabela de clientes são Parana e Para, ambos com 7 ocorrências. Em seguida, vêm Rondônia com 6 ocorrências, e vários estados com 5 ocorrências, incluindo Tocantins, Sergipe, Rio de Janeiro, Maranhão, Goiás, Ceará e Amapá. Esses resultados mostram a distribuição geográfica dos clientes na base de dados.


In [14]:
print(response.metadata)

{'22a16373-24ce-48ef-a840-de80d01d49e7': {'sql_query': 'SELECT Estado, COUNT(ID_Cliente) AS Frequencia FROM Clientes GROUP BY Estado ORDER BY Frequencia DESC', 'result': [('parana', 7), ('para', 7), ('rondonia', 6), ('tocantins', 5), ('sergipe', 5), ('rio de janeiro', 5), ('maranhao', 5), ('goias', 5), ('ceara', 5), ('amapa', 5), ('roraima', 4), ('rio grande do norte', 4), ('piaui', 4), ('pernambuco', 4), ('distrito federal', 4), ('sao paulo', 3), ('minas gerais', 3), ('mato grosso do sul', 3), ('bahia', 3), ('santa catarina', 2), ('rio grande do sul', 2), ('paraiba', 2), ('mato grosso', 2), ('amazonas', 2), ('espirito santo', 1), ('alagoas', 1), ('acre', 1)], 'col_keys': ['Estado', 'Frequencia']}, 'sql_query': 'SELECT Estado, COUNT(ID_Cliente) AS Frequencia FROM Clientes GROUP BY Estado ORDER BY Frequencia DESC', 'result': [('parana', 7), ('para', 7), ('rondonia', 6), ('tocantins', 5), ('sergipe', 5), ('rio de janeiro', 5), ('maranhao', 5), ('goias', 5), ('ceara', 5), ('amapa', 5), ('

In [15]:
response = query_engine.query('Quais as fornecedoras localizadas na cidadde de São Paulo?')
print(response)

Não há fornecedoras localizadas na cidade de São Paulo. A busca não retornou resultados.


In [16]:
response = query_engine.query('Quais funcionarios localizados na cidade de São Paulo?')
print(response)

Não há funcionários localizados na cidade de São Paulo. A consulta ao banco de dados não retornou resultados.


# Contextualizing model

In [17]:
llm = Groq(model=llama_model, api_key=GROQ_API_KEY)

def generate_table_description(table_name, sample_df):
    prompt = f'''
    Analyze the sample of the table '{table_name}' below and give a short and concise description of the table contents.
    Inform up to 5 unique values of each column.

    Table sample:
    {sample_df}

    Description:
    '''

    response = llm.complete(prompt)

    return response.text

In [18]:
tables_names = metadata_obj.tables.keys()
tables_dict = {}

for table_name in tables_names:
    df = pd.read_sql_table(table_name, engine)
    sample_df = df.head(5).to_string()

    description = generate_table_description(table_name, sample_df)
    tables_dict[table_name] = description

    print(f'Table: {table_name}\nDescription:\n{description}')
    print('-'*15)

Table: Clientes
Description:
The 'Clientes' table contains information about clients, including their identification, contact details, address, and spending history. 

Here are up to 5 unique values for each column:

- ID_Cliente: 1, 2, 3, 4, 5
- Nome: lucas moura, julia andrade, rafael dias, carla souza, felipe neto
- Email: lucas.moura@email.com, julia.andrade@email.com, rafael.dias@email.com, carla.souza@email.com, felipe.neto@email.com
- Rua_Numero: sitio de cardoso, 49, trecho mariane teixeira, 90, setor de duarte, 114, recanto ana livia lopes, 53, jardim de monteiro, 11
- Estado: sao paulo, tocantins, paraiba, para
- Valor_gasto: 29017, 7834, 4071, 17512, 24307
---------------
Table: Fornecedores
Description:
The table 'Fornecedores' appears to contain information about suppliers or vendors. It includes details such as unique identifiers, names, contact persons, phone numbers, email addresses, and physical addresses.

Here are up to 5 unique values for each column:

1. ID_Fornece

In [19]:
table_schema_obj = [
    SQLTableSchema(table_name=table_name,context_str=tables_dict[table_name])
    for table_name in tables_names
]

In [20]:
obj_index = ObjectIndex.from_objects(table_schema_obj, table_node_map, VectorStoreIndex)
obj_retriever = obj_index.as_retriever(similarity_top_k=1)

In [21]:
query_engine2 = SQLTableRetrieverQueryEngine(sql_database=sql_dtabase, table_retriever=obj_retriever)

In [22]:
response = query_engine2.query('Quais os estados mais frequentes na tabela de clientes?')
print(response)

Os estados mais frequentes na tabela de clientes são Parana e Para, ambos com 7 ocorrências. Em seguida, vêm Rondônia com 6 ocorrências, e vários estados com 5 ocorrências, incluindo Tocantins, Sergipe, Rio de Janeiro, Maranhão, Goiás, Ceará e Amapá. Esses resultados mostram a distribuição geográfica dos clientes na base de dados.


In [23]:
response = query_engine2.query('Quais as fornecedoras localizadas na cidadde de São Paulo?')
print(response)

As fornecedoras localizadas na cidade de São Paulo são:

1. Technex Solutions, localizada na Rua das Inovações, 123, no bairro do Centro.
2. Nexustech Systems, localizada na Travessa da Conexão, 567, no bairro do Parque das Maravilhas.

Essas são as duas fornecedoras que atendem ao critério de estar localizadas na cidade de São Paulo, de acordo com os dados disponíveis.


In [24]:
response = query_engine2.query('Quais funcionarios localizados na cidade de São Paulo?')
print(response)

Os funcionários localizados na cidade de São Paulo são: 

1. Carla Souza - Vendas
2. Paulo Silva - Vendas
3. Marta Rocha - Gerência
4. Sofia Ramos - Técnico
5. Joao Pereira - Estoque
6. Talita Borges - Financeiro
7. Larissa Freitas - Vendas

Esses são os funcionários que atuam na cidade de São Paulo, de acordo com os cargos especificados.


## Prompts personalization

In [25]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
print(DEFAULT_TEXT_TO_SQL_PROMPT.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [26]:
from llama_index.core import PromptTemplate

prompt = '''
Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 
'''

prompt_1 = PromptTemplate(prompt, dialect = engine.dialect.name)

### Table Descriptions

In [27]:
from typing import List

def table_description(tables_schema: List[SQLTableSchema]):
    descriptions = []
    for table_schema in tables_schema:
        table_info = sql_dtabase.get_single_table_info(table_schema.table_name)
        table_info += (' The table\'s description is: ' + table_schema.context_str) 

        descriptions.append(table_info)
    return '\n\n'.join(descriptions)


In [28]:
from llama_index.core.query_pipeline import FnComponent

table_context = FnComponent(fn=table_description)

In [29]:
from llama_index.core.llms import ChatResponse

def sql_response(response: ChatResponse) -> str:
    response_content = response.message.content

    sql_query = response_content.split("SQLQuery: ", 1)[-1].split("SQLResult: ", 1)[0]

    return sql_query.strip().strip('```').strip()


sql_query = FnComponent(fn=sql_response)

In [30]:
from llama_index.core.retrievers import SQLRetriever

sql_result = SQLRetriever(sql_database=sql_dtabase)

In [31]:
prompt_2_str = '''
You are a Database query assistant.
Given the follow question, the SQL query and the SQL result, respond the user question in a nice way and be objective.
Avoid to start the conversation with apresentations or salutations, like "Hello".capitalize

Question: {user_question}
SQL query: {query}
SQL result: {result}
Response:
'''

prompt_2 = PromptTemplate(prompt_2_str)

In [32]:
from llama_index.core.query_pipeline import QueryPipeline, InputComponent

qp = QueryPipeline(
    modules={
        "input": InputComponent(),
        "table_access": obj_retriever,
        "table_context": table_context,
        "prompt_1": prompt_1,
        "llm_1": llm,
        "sql_query": sql_query,
        "sql_result": sql_result,
        "prompt_2": prompt_2,
        "llm_2": llm
    },
    verbose=False
)

In [33]:
qp.add_chain(['input', 'table_access', 'table_context'])
qp.add_link('input', 'prompt_1', dest_key='query_str')
qp.add_link('table_context', 'prompt_1', dest_key='schema')

qp.add_chain(['prompt_1', 'llm_1', 'sql_query', 'sql_result'])
qp.add_link('input', 'prompt_2', dest_key='user_question')
qp.add_link('sql_query', 'prompt_2', dest_key='query')
qp.add_link('sql_result', 'prompt_2', dest_key='result')
qp.add_link('prompt_2', 'llm_2')


### Testing the outputs

In [34]:
response = qp.run(query="Quais os estados mais frequentes na tabela de Clientes?")
print(str(response))

assistant: Os estados mais frequentes na tabela de Clientes são Parana e Para, ambos com 7 ocorrências. Em seguida, aparece Rondonia com 6 ocorrências. É importante notar que há vários estados com a mesma frequência, como Tocantins, Sergipe, Rio de Janeiro, Maranhao, Goias, Ceara e Amapa, todos com 5 ocorrências. Além disso, Roraima, Rio Grande do Norte, Piaui e Pernambuco têm 4 ocorrências cada, enquanto Distrito Federal também tem 4 ocorrências. Os estados com menor frequência são Espirito Santo, Alagoas e Acre, com apenas 1 ocorrência cada.


In [35]:
print(str(response))

assistant: Os estados mais frequentes na tabela de Clientes são Parana e Para, ambos com 7 ocorrências. Em seguida, aparece Rondonia com 6 ocorrências. É importante notar que há vários estados com a mesma frequência, como Tocantins, Sergipe, Rio de Janeiro, Maranhao, Goias, Ceara e Amapa, todos com 5 ocorrências. Além disso, Roraima, Rio Grande do Norte, Piaui e Pernambuco têm 4 ocorrências cada, enquanto Distrito Federal também tem 4 ocorrências. Os estados com menor frequência são Espirito Santo, Alagoas e Acre, com apenas 1 ocorrência cada.


# Interface with Gradio

In [36]:
def input_output(user_message: str):
    response = qp.run(query=user_message)
    return str(response.message.content)

In [37]:
def add_to_history(user_message: str, history):
    assistant_message = input_output(user_message)
    history = history or []
    history.append([user_message, assistant_message])
    return assistant_message, history

In [38]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## Database Query Assistant")
    gr.Markdown("""Ask anything about the database. 
                You can query information about Clientes (customers), Fornecedores (suppliers), and Funcionarios (employees). 
                Try questions like 'Quais os estados mais frequentes na tabela de Clientes?' or 'Mostre os fornecedores com seus contatos'.""")
    
    chatbot = gr.Chatbot(label="Chat with the database")
    msg = gr.Textbox(label="Enter a message and press enter", placeholder="Enter a message")
    clear = gr.ClearButton([msg, chatbot])
    
    def update_history(user_message, history):
        _, history = add_to_history(user_message, history)
        return "", history

    msg.submit(update_history, inputs=[msg, chatbot], outputs=[msg, chatbot], queue=False)
    clear.click(lambda: None, inputs=None, outputs=[chatbot], queue=False)

demo.queue()
demo.launch(debug=True)
        

/tmp/ipykernel_418166/2358672485.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat with the database")


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
